# Advanced Feature Engineering I
<hr style="border:2px solid black">

## Penguin Data

**load packages**

In [ ]:
# data analysis stack
import numpy as np
import pandas as pd

# machine-learning stack
from sklearn.metrics import r2_score  
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    MinMaxScaler,
    PolynomialFeatures
)
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline

# miscellaneous
import warnings
warnings.filterwarnings("ignore")

from sklearn import set_config
set_config(transform_output='pandas')

## Business Goal:
>Predict the body mass based on the other feature

**read data**

In [ ]:
df = pd.read_csv('./data/penguins.csv')
df

## Select columns for X and y

In [ ]:
# keep a copy 
df_copy = df.copy()

In [ ]:
X = df_copy.drop(['body_mass_g'],axis=1)
X_ash = df_copy.drop(columns=['body_mass_g'])
y = df_copy["body_mass_g"]

### Train-Test split

In [ ]:
RSEED = 32

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y, random_state= RSEED, shuffle=True, test_size=0.2)
Xtrain.shape, ytrain.shape

## EDA
- which variable has missing values?
- which variables are binary, categorical, metric?
- do categorical variables have non-numeric values?
- do metric features are varying on a different scale?
- ...

In [ ]:
# Combine back Xtrain, ytrain
df_train = pd.concat([Xtrain,ytrain],axis=1)
df_train

In [ ]:
# check for missing value 

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train

In [ ]:
# check for correlation


<hr style="border:2px solid black">

## Feature Engineering

- `Pipeline` combines ML steps in series
- `ColumnTransformer` combines ML steps in parallel

## Preprocessing


Which colum we want to transform:
+ sex (categorical nominal) --> imputation, encoding
+ flipper lenght (numerical continuous) --> imputation (median or mean), then scaling or binning
+ bill depth (numerical continuous) --> imputation (median or mean), then scaling or binning
+ bill lenght (numerical continuous) --> scaling or binning


In [ ]:
# instantiate the transformation
sex_imputer = SimpleImputer(strategy='most_frequent')

In [ ]:
sex_imputer.fit(Xtrain[['sex']])

In [ ]:
# get the most frequent
sex_imputer.statistics_

In [ ]:
# applay the transformation
sex_col_imputed = sex_imputer.transform(Xtrain[['sex']])
sex_col_imputed

In [ ]:
# instantiate the transformation
ohe = OneHotEncoder(drop='first', sparse_output=False)

In [ ]:
ohe.fit(sex_col_imputed[['sex']])

In [ ]:
ohe.fit(sex_col_imputed) # the same as the previous cell 

In [ ]:
# getting the categories
ohe.categories_

In [ ]:
imputed_ohe = ohe.transform(sex_col_imputed[['sex']])
imputed_ohe

In [ ]:
sex_transf = Pipeline(steps=[
    ('sex_imputation', SimpleImputer(strategy='most_frequent')),
    ('sex_ohe', OneHotEncoder(drop='first', sparse_output=False))
])

In [ ]:
sex_transf.fit(Xtrain[['sex']])

In [ ]:
pipeline_sex_encoded_imputed = sex_transf.transform(Xtrain[['sex']])
pipeline_sex_encoded_imputed

In [ ]:
sex_transf.get_feature_names_out()

In [ ]:
#'bill_length_mm'
scaler = StandardScaler()

In [ ]:
scaler.fit(Xtrain[['bill_length_mm']])

In [ ]:
scaler.mean_, scaler.scale_

In [ ]:
trasformed_bill_lenght = scaler.transform(Xtrain[['bill_length_mm']],)
trasformed_bill_lenght

In [ ]:
scaler.get_feature_names_out()

In [ ]:
Xtrain_fe = pd.concat([trasformed_bill_lenght,pipeline_sex_encoded_imputed],axis=1)
Xtrain_fe

In [ ]:
sex_transf

In [ ]:
scaler

In [ ]:
Xtrain.isna().sum()

### ColumnTransfomer

In [ ]:
fe_transformer = ColumnTransformer(transformers=[
    ('transf_cat', sex_transf, ['sex', 'island'] ),
    ('bill_lenght', scaler , ['bill_length_mm'])
],remainder='drop')
fe_transformer

In [ ]:
fe_transformer.fit(Xtrain)

In [ ]:
X_train_transformed = fe_transformer.transform(Xtrain)
X_train_transformed

In [ ]:
X_test_transformed = fe_transformer.transform(Xtest)
X_test_transformed

### Model Building

**instantiate model**

In [ ]:
model = LinearRegression()

**train model**

In [ ]:
model.fit(X_train_transformed,ytrain)

### Model Evaluation

**model performance**

In [ ]:
print('r2 on train',model.score(X_train_transformed,ytrain))
print('r2 on test',model.score(X_test_transformed,ytest))

In [ ]:
y_predict_train = model.predict(X_train_transformed)
y_predict_test = model.predict(X_test_transformed)

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
plt.scatter(y_predict_train, ytrain)
plt.xlabel('ypred')
plt.ylabel('ytrue')

## References

- [How to add feature engineering to a scikit-learn pipeline](https://practicaldatascience.co.uk/machine-learning/how-to-add-feature-engineering-to-a-scikit-learn-pipeline)

- [Coding a custom imputer in scikit-learn](https://towardsdatascience.com/coding-a-custom-imputer-in-scikit-learn-31bd68e541de)

<hr style="border:2px solid black">

#### Doing featuring eng and modeling in one go

In [ ]:
Pipeline(steps=[
    ('fe_engineering',fe_transformer),
    ('linear_regression', LinearRegression())
])

In [ ]:
linear_model_one_line = Pipeline(steps=[
    ('fe_engineering',fe_transformer),
    ('linear_regression', LinearRegression())
])
linear_model_one_line.fit(Xtrain,ytrain);

In [ ]:
linear_model_one_line.score(Xtest,ytest)

In [ ]:
linear_model_one_line.predict(Xtest)

# Exercise: engineering other features